# Actualizacion de Tablas SQL
## Utilizando las nuevas tablas desde el modulo de sistematizacion

Antes que todo, es necesario que la carpeta `db/CSV/` se encuentra con los archivos nuevos -i.e., copiar y reemplazar en la carpeta las tablas nuevas.

En particular, las nuevas tablas contienen procesada la columna `macro`. Esta columna asigna una etiqueta o categoria, permitiendo agrupar distintas respuestas. 

In [11]:
import pandas as pd
import numpy as np
import os 


from use_cases.pairs import create_pair_token
from use_cases.utils.textools import clean_alt_list, to_unicode

from use_cases.contributions import create_table_contributions, to_sql as con_to_sql
from use_cases.emotions import create_table_emotions, to_sql as emo_to_sql 
from use_cases.personal_needs import create_table_personal_needs, to_sql as per_need_to_sql
from use_cases.country_needs import create_table_country_needs, to_sql as count_need_to_sql

### Emociones y Sentimientos

In [40]:
emotions_table = pd.read_csv('./db/CSV/emotions.csv', low_memory=False)

In [41]:
emotions_table.sample(2)

,id,diag_id,ind_id,name,name_tokens,macro,exp,exp_tokens,is_online
49562,49562,ENC_U_4554494271909703782,NaN,alegria,['alegria'],Alegría,-por el movimiento social y por la unidad que ...,"['movimiento', 'social', 'unidad', 'movimiento...",True
77688,77688,NaN,3228981,rechazo,['rechazo'],Repulsión,NaN,NaN,False


las listas se guardan como `strings` en el csv, es necesario volver a dejarlos como `listas`, asi tambien como pasar los id de los dialogos a `unicode`

In [45]:
emotions_table['name_tokens'] = emotions_table['name_tokens'].apply(lambda x: clean_alt_list(x))
emotions_table['exp_tokens'] = emotions_table['exp_tokens'].apply(lambda x: clean_alt_list(x))
emotions_table = emotions_table.replace({'nr':'','nan':'', 'NR':'', 'NaN':'', np.nan:''})
emotions_table['diag_id'] = to_unicode(emotions_table['diag_id'])
emotions_table = emotions_table.drop(emotions_table[(emotions_table['diag_id'] == '') & (emotions_table['ind_id'] == '')].index) 

In [46]:
emotions_table.sample(2)

,id,diag_id,ind_id,name,name_tokens,macro,exp,exp_tokens,is_online
60179,60179,enc_u_4562081741901988839,,.,[nan],,.,[nan],True
1158,1158,enc_u_4557021980801366643,,decepcionado,[decepcionado],Enojo,por el actuar de las autoridades tras no apoya...,"[actuar, autoridades, tras, apoyar, fu...",True


Debemos recalcular los pares de palabras en caso de que algo haya cambiado

In [47]:
emotion_pair = create_pair_token(emotions_table, 'exp_tokens', 'emotion_id')

guardamos/reemplazamos la nueva tabla de pares 

In [48]:
emotion_pair.to_csv('./db/CSV/emotions_pairs.csv', index=False)

y finalmente los nuevos `.sql`

In [49]:
emo_to_sql(emotions_table, './db/SQL_INSERT/emotions.sql')

### Contribuciones

En el caso de contribuciones solo necesitamos guardar el nuevo `csv` como `sql`

In [54]:
contributions_table = pd.read_csv('./db/CSV/contributions.csv', low_memory=False)

In [55]:
contributions_table.sample(1)

,id,diag_id,ind_id,text,tokens,macro,is_online
64540,79629,NaN,3629873,"votar, para que este pais sea mejor","['votar', 'pais', 'mejor']",Participacion,False


In [56]:
contributions_table['tokens'] = contributions_table['tokens'].apply(lambda x: clean_alt_list(x))
contributions_table = contributions_table.replace({'nr':'','nan':'', 'NR':'', 'NaN':'', np.nan:''})
contributions_table['diag_id'] = to_unicode(contributions_table['diag_id'])
contributions_table = contributions_table.drop(contributions_table[(contributions_table['diag_id'] == '') & (contributions_table['ind_id'] == '')].index)

In [57]:
contributions_table.sample(1)

,id,diag_id,ind_id,text,tokens,macro,is_online
54305,66674,,2810793,soy una adulta mayor. mi aporte a mi pais ya l...,"[adulta, mayor, aporte, pais, di, trabaje...",Reciprocidad-Redes,False


In [58]:
con_to_sql(contributions_table, './db/SQL_INSERT/contributions.sql')